Auto-Fetching/Caching:

# Nỗi đau khi không dùng RTK Query:
- Hãy tưởng tượng luồng hoạt động truyền thống:
    1. Bạn vào trang chủ, gọi API lấy danh sách sản phẩm (VD: có 10 sản phẩm)
    2. Bạn bấm nút "Thêm mới" để thêm sản phẩm thứ 11
    3. API thêm thành công. Modal đóng lại
    4. VẤN ĐỀ: Danh sách ở trang chủ vẫn hiển thị 10 sản phẩm cũ
=> Giải pháp thủ công: 
    - Phải tự viết code để gọi lại API danh sách
    - Hoặc refresh lại trang (F5) thì mới cập nhật danh sách mới nhất

# Auto-Fetching/Caching (Tự động tải lại) sinh ra để giải quyết vấn đề đó:
>> RTK Query giúp bạn:
- Ngay sau khi THÊM/SỬA/XOÁ xong -> danh sách tự động tải lại mà không cần viết thêm dòng code nào ở component
- Nó dùng cơ chế Tags (Nhãn dán). Hãy tưởng tượng như hệ thống thông báo:
    - **Caching (Bộ nhớ đệm)**: Khi tải danh sách về, RTK Query lưu nó lại và dán cho nó cãi nhãn (VD: nhãn tên là `Products`)

    - **Invalidating (Báo cũ/Huỷ hiệu lực)**: Khi bạn THÊM/SỬA/XOÁ, bạn bảo với RTK Query:
        >> "Ê, dữ liệu liên quan đến cái nhãn `Products` bị thay đổi rồi đấy, CÁI CŨ HỎNG RỒI!!"
                
    - **Auto Fetching**: RTK Query nghe thấy vậy -> nó lập tức đi tìm xem ở đâu đang dùng nhãn `Products` và tự động chạy lại API đó để lấy dữ liệu mới nhất

In [ ]:
// 3. Cách dùng trong Code (src//service/product.js)
// Chúng ta cần cấu hình 3 bước trong `createApi`

/* Bước 1: Khai báo tên các loại nhãn --> `tagTypes` */
export const productApi = createApi({
    reducerPath: "productApi",
    baseQuery,
    tagTypes: ["Products"], // 1. Khai báo: Tôi có dùng nhãn tên là "Products"
    endpoints: (builder) => ({
        // ...
    }),
});

In [ ]:

/* Bước 2: Dán nhãn cho API lấy dữ liệu (GET) --> `providesTag` */
getProducts: builder.query({
    query: (page) => `/products?page=${page}`,
    // 2. Dán nhãn: Kết quả trả về của API này thuộc nhóm "Products"
    providesTags: ["Products"], 
})


In [ ]:

/* Bước 3: Báo hiệu lực thay đổi --> `invalidatesTags` */
createProduct: builder.mutation({
    query: (body) => ({
        url: "/products",
        method: "POST",
        body,
    }),
    // 3. Báo cũ: Sau khi thêm xong, dữ liệu nhóm "Products" đã bị cũ (invalid).
    // => RTK Query hãy tự động chạy lại thằng nào đang giữ nhãn "Products" (chính là getProducts ở trên).
    invalidatesTags: ["Products"], 
})


TÓM LẠI:
1. `tagTypes`: Danh sách tên các nhãn
2. `providesTags`: Gán nhãn cho dữ liệu tải về (Thường dùng ở `query`)
3. `invalidatesTags`: Báo cho hệ thống biết nhãn nào cần tải lại (Thường dùng ở `mutation`).
